In [1]:
import pyspark

try: 
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')    
    
type(sc)

pyspark.context.SparkContext

1) A partir de la plataforma online (e-shop) de los países en los que opera, Nintendo tiene información de ventas de videojuegos diarias digitales por país en el siguiente RDD:  
(id_videojuego, codigo_pais, fecha, visitas_diarias, total_ventas_diarias).  
Por otro lado se tienen otro RDD que tiene información de todos los videojuegos que se venden en su plataforma con el siguiente formato  
(id_videojuego, titulo, rating_pegi, rating_esbr).  
Tener en cuenta que un mismo videojuego se puede vender en distintos países y esos nos permitirá obtener métricas a nivel global. Con esta información escribir un programa en pySpark que permita:  
a) Obtener el videojuego con más ventas digitales globales (es decir en todos los países) en un RDD con el siguiente formato: (id_videojuego, titulo, total), siendo total la cantidad total de ventas digitales globales  
b) Para el videojuego con mas ventas, obtener cual es el país para el cual ser registra una mayor tasa de conversión (es decir, mayor total_ventas_diarias / visitas_diarias) (**) (15 pts)

In [2]:
ventas = [(1, 32, '20171201', 1024, 99),
          (3, 32, '20151201', 24, 20),
          (1, 32, '20171203', 4410, 325),
          (2, 32, '20161204', 58, 31),
          (2, 36, '20161204', 254, 102),
          (3, 36, '20171210', 234, 129),
          (1, 380, '20171211', 10624, 8979)]
juegos = [(1, 'Another World', '16', 'Mature'),
          (2, 'Maniac Mansion', '12', 'Teen'),
          (3, 'Bubble Bubble', '3', 'Everyone')]
paises = [(32, 'Argentina'),
          (36, 'Australia'),
          (380, 'Italia')]

rddVentas = sc.parallelize(ventas)
rddJuegos = sc.parallelize(juegos)
rddPaises = sc.parallelize(paises)

In [3]:
# Solo para ver como va quedando
rddVentas.map(lambda x: (x[0],x[4]))\
.collect()

[(1, 99), (3, 20), (1, 325), (2, 31), (2, 102), (3, 129), (1, 8979)]

In [4]:
# Solo para ver como va quedando
rddVentas.map(lambda x: (x[0],x[4]))\
.reduceByKey(lambda x,y: x+y)\
.collect()

[(2, 133), (1, 9403), (3, 149)]

In [5]:
# Solo para ver como va quedando
rddVentas.map(lambda x: (x[0],x[4]))\
.reduceByKey(lambda x,y: x+y)\
.join(rddJuegos.map(lambda x: (x[0],x[1])))\
.map(lambda x: (x[0],x[1][1],x[1][0]))\
.collect()

[(1, 'Another World', 9403),
 (2, 'Maniac Mansion', 133),
 (3, 'Bubble Bubble', 149)]

a) Obtener el videojuego con más ventas digitales globales (es decir en todos los países) en un RDD con el siguiente formato: (id_videojuego, titulo, total), siendo total la cantidad total de ventas digitales globales

In [6]:
# Solucion parte a
rdd_a = rddVentas.map(lambda x: (x[0],x[4]))\
.reduceByKey(lambda x,y: x+y)\
.join(rddJuegos.map(lambda x: (x[0],x[1])))\
.map(lambda x: (x[0],x[1][1],x[1][0]))
tupla = rdd_a.reduce(lambda x,y: x if x[2]>y[2] else y)
tupla

(1, 'Another World', 9403)

b) Para el videojuego con mas ventas, obtener cual es el país para el cual ser registra una mayor tasa de conversión (es decir, mayor total_ventas_diarias / visitas_diarias) (**) (15 pts)

In [7]:
# Solucion parte b
rddVentas.map(lambda x: (x[0],(x[1],x[3],x[4])))\
.filter(lambda x: x[0] == tupla[0])\
.map(lambda x: (x[1][0],(x[1][1],x[1][2])))\
.reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1]))\
.map(lambda x: (x[0],(float(x[1][1])/float(x[1][0]))))\
.reduce(lambda x,y: x if x[1]>y[1] else y)

(380, 0.8451618975903614)

### Otra forma, más parecida a como haciamos ayer

In [8]:
# Solo para ver como va quedando
rddVentasById = rddVentas.map(lambda x: (x[0],(x[1],x[4])))
rddJuegosById = rddJuegos.map(lambda x: (x[0],(x[1])))

rddVentasById.join(rddJuegosById)\
.collect()

[(1, ((32, 99), 'Another World')),
 (1, ((32, 325), 'Another World')),
 (1, ((380, 8979), 'Another World')),
 (2, ((32, 31), 'Maniac Mansion')),
 (2, ((36, 102), 'Maniac Mansion')),
 (3, ((32, 20), 'Bubble Bubble')),
 (3, ((36, 129), 'Bubble Bubble'))]

In [9]:
# Solo para ver como va quedando
rddVentasById.join(rddJuegosById)\
.map(lambda x: (x[0],(x[1][0][0],x[1][0][1],x[1][1])))\
.collect()

[(1, (32, 99, 'Another World')),
 (1, (32, 325, 'Another World')),
 (1, (380, 8979, 'Another World')),
 (2, (32, 31, 'Maniac Mansion')),
 (2, (36, 102, 'Maniac Mansion')),
 (3, (32, 20, 'Bubble Bubble')),
 (3, (36, 129, 'Bubble Bubble'))]

a) Obtener el videojuego con más ventas digitales globales (es decir en todos los países) en un RDD con el siguiente formato: (id_videojuego, titulo, total), siendo total la cantidad total de ventas digitales globales

In [10]:
# Solucion parte a
rddVentasById.join(rddJuegosById)\
.map(lambda x: (x[0],(x[1][0][0],x[1][0][1],x[1][1])))\
.reduceByKey(lambda x,y: (x[0], (x[1]+y[1]), x[2]))\
.map(lambda x: (x[0], x[1][2], x[1][1]))\
.reduce(lambda x,y: x if x[2]>y[2] else y)

(1, 'Another World', 9403)